In [50]:
from codebase.classes_ibis_lvm import ParticlesLVM
from codebase.classes_data import Data
from codebase.ibis import model_phonebook, essl
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_stan_model, remove_chain_dim
from run_ibis_lvm import run_ibis_lvm
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
from codebase.plot import plot_density, get_post_df, plot_correlations, plot_line
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## 2 factor Sim EZ

In [12]:
existing_directory = None
task_handle = 'ez_lvm'
gen_model = 0

if existing_directory is None:
    log_dir = make_folder(task_handle)  
    print("\n\nCreating new directory: %s" % log_dir)

else:
    log_dir = existing_directory
    log_dir = path_backslash(log_dir)
    print("\n\nReading from existing directory: %s" % log_dir)




Creating new directory: ./log/20210201_220050_ez_lvm/


## Create Data

In [13]:
# generate data
exp_data = Data(
    name = task_handle, 
    model_num = 1, 
    size = 50,
    random_seed = 6
    )
    
exp_data.generate()

In [15]:
# particles = ParticlesLVM(
#     name = 'ibis_lvm',
#     model_num = 7,
#     size = 100,
#     bundle_size=50,
#     param_names = ['param_names'],
#     latent_names = ['latent_names'],
#     latent_model_num= 1
# )
# particles.set_log_dir(log_dir)
# # particles.compile_model()
# particles.compile_prior_model()


## Run HMC

In [16]:
# sm = load_obj('sm', 'log/compiled_models/model8/')

import pystan

with open(
    'codebase/stancode/models/hmc/model_7.stan',
    'r'
    ) as file:
    model_code = file.read()

    sm = pystan.StanModel(model_code=model_code, verbose=False)
save_obj(sm, 'sm_hmc', log_dir)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cac0b7f4994c30bbd65d089b1cb38865 NOW.


In [17]:
# sm = load_obj('sm', 'log/compiled_models/model8/')
sm = load_obj('sm_hmc', log_dir)

fit_run = run_stan_model(
    data = exp_data.get_stan_data(),
    compiled_model = sm,
    num_samples = 1000, 
    num_warmup = 1000,
    num_chains = 1,
#     initial_values={'beta':[1,1,1,1,1,1]},
    adapt_engaged=True,
    )

In [19]:
param_names = ['beta', 'alpha']

ps = fit_run.extract(
        permuted=False, pars=param_names)

save_obj(ps, 'ps_hmc', log_dir)

In [94]:
ps = load_obj('ps_hmc', log_dir)

In [95]:
nsim = ps['beta'].shape[0]
ps = remove_chain_dim(ps, param_names, num_samples=nsim)


In [96]:
param = 'beta'
df = get_post_df(ps[param]) 
df['source'] = 'mcmc'
plot_line(df, height=100)

alt.Chart(...)

In [97]:
# nrows = ps['beta'].shape[1]
# for n in range(nsim):
#     for i in range(nrows):
#         sign = np.sign(ps['beta'][n,0])
#         ps['beta'][n] = sign * ps['beta'][n]

In [98]:
param = 'beta'
df = get_post_df(ps[param]) 
df['source'] = 'mcmc'
plot_density(df, height=100)

alt.Chart(...)

## Run SMC^2

In [87]:
model_num = 7
gen_model = False


ibis = run_ibis_lvm(
    exp_data,
    model_num,
    500,
    100,
    gen_model,
    log_dir
    )


100%|██████████| 50/50 [46:21<00:00, 55.62s/it] 




Marginal Likelihood 1.45058


In [5]:
# load existing results

# log_dir = 'log/20201218_172340_ez_comp/'
# particles = load_obj('particles', log_dir)

## Plot Both results

In [99]:
# have to resample particles to get rid of weights
particles = ibis['particles']
particles.resample_particles_bundles()
# particles.jitter(exp_data.get_stan_data_upto_t(exp_data.size))

In [100]:
ps_smc2 = dict()
samples = np.empty((particles.size, 1, 6))
for i in range(particles.size):
    samples[i] = particles.particles[i].particles['alpha']
ps_smc2['alpha'] = samples
samples = np.empty((particles.size, 6, 1))
for i in range(particles.size):
    samples[i] = particles.particles[i].particles['beta']
ps_smc2['beta'] = samples

In [101]:
ps_smc2['alpha'].shape

(500, 1, 6)

In [102]:
param = 'beta'
df = get_post_df(ps[param]) 
df['source'] = 'hmc'
df2 = get_post_df(ps_smc2[param])
df2['source'] = 'smc2'
plot_density(pd.concat([df,df2])) 
# &\
# plot_correlations(ibis['jitter_corrs'][param], height = 150)

alt.Chart(...)

In [103]:
ps[param].shape

(1000, 6, 1)

In [104]:
param = 'alpha'
df = get_post_df(ps[param]) 
df['source'] = 'hmc'
df2 = get_post_df(ps_smc2[param])
df2['source'] = 'smc2'
plot_density(pd.concat([df,df2])) 

ValueError: Must pass 2-d input